In [35]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [29]:
features = pd.merge(
    pd.read_csv('data/X_train.csv'), 
    pd.read_csv('data/y_train.csv'), 
    on=['series_id']
)

In [130]:
max_group = features['group_id'].max()
max_series = features['series_id'].max()
slices = []
for group in features.groupby('group_id').groups.values():
    group = features.iloc[group]
    for i in range(len(group)/128 - 1):
        new_slice = group[64+i*128:64+(i+1)*128].reset_index(drop=True)
        new_slice['measurement_number'] = new_slice.index
        max_series += 1
        max_group += 1
        new_slice['series_id'] = max_series
        new_slice['group_id'] = max_group
        new_slice['row_id'] = new_slice[['series_id', 'measurement_number']].apply(lambda x: '{}_{}'.format(x[0], x[1]), axis=1)

        slices.append(new_slice)
    
slices = pd.concat(slices, ignore_index=True).reset_index(drop=True)
    

In [140]:
augmented_features = pd.concat([features, slices],ignore_index=True).reset_index(drop=True)
augmented_features.drop(['group_id', 'surface'], axis=1).to_csv('data/X_train_augment.csv', index=False)
augmented_features[['series_id', 'group_id', 'surface']].drop_duplicates().to_csv('data/y_train_augment.csv', index=False)
